## Transcrire un algorithme (Grover)

### Registres

In [1]:
from qiskit import QuantumRegister, ClassicalRegister

valeurs = QuantumRegister(3, 'valeurs')
mesures = ClassicalRegister(valeurs.size, 'mesures')

### Oracle

In [2]:
from qiskit import QuantumCircuit

oracle = QuantumCircuit(valeurs.size)
oracle.barrier(range(oracle.num_qubits))
oracle.x(1)
oracle.ccz(0, 1, 2)
oracle.x(1)
oracle.barrier(range(oracle.num_qubits))
oracle.draw()

### Diffuseur

In [3]:
diffuseur = QuantumCircuit(valeurs.size)
diffuseur.h(range(diffuseur.num_qubits))
diffuseur.x(range(diffuseur.num_qubits))
# diffuseur.barrier(range(diffuseur.num_qubits-1))
diffuseur.h(diffuseur.num_qubits-1)
diffuseur.ccx(0, 1, 2)
diffuseur.h(diffuseur.num_qubits-1)
# diffuseur.barrier(range(diffuseur.num_qubits-1))
diffuseur.x(range(diffuseur.num_qubits))
diffuseur.h(range(diffuseur.num_qubits))
diffuseur.draw()

### Circuit

#### Amorce
Portes d'Hadamard + Oracle

In [4]:
circuit = QuantumCircuit(valeurs)
circuit.h(range(circuit.num_qubits))
circuit = circuit.compose(oracle)
circuit.draw()

#### Vérification de l'oracle
Simulation partielle

In [9]:
# Qiskit v0.46.1
#  from qiskit import transpile, BasicAer

#  simulator = BasicAer.get_backend('statevector_simulator')
#  result = simulator.run(transpile(circuit, simulator), shots=256).result()
#  states = result.get_statevector(circuit)

# Qiskit v1.0.2
from qiskit.quantum_info import Statevector

states = Statevector(circuit)

Phases

In [10]:
from numpy import arctan2, degrees, round

[round(degrees(arctan2(state.imag, state.real))) for state in states]

Graphique

In [11]:
from qiskit.visualization import plot_state_qsphere

plot_state_qsphere(states, show_state_phases=True)

#### Ajout du diffuseur

In [12]:
circuit.compose(diffuseur, inplace=True)
circuit.draw()

#### Répétition + Mesure

In [13]:
circuit = circuit \
    .compose(oracle) \
    .compose(diffuseur)

circuit.add_register(mesures)
circuit.measure_all(add_bits=False)
circuit.draw()

### Simulation

In [14]:
# Qiskit v0.46.1
#  from qiskit import transpile, BasicAer
#  simulator = BasicAer.get_backend('qasm_simulator')

from qiskit import transpile
from qiskit.visualization import plot_histogram
from qiskit.providers.basic_provider import BasicProvider

simulator = BasicProvider().get_backend('basic_simulator')
compiled = transpile(circuit, simulator)
job = simulator.run(compiled, shots=1000)
result = job.result()

counts = result.get_counts(compiled).int_outcomes()
plot_histogram(counts)